In [1]:
import sys
import os

path = os.path.dirname(os.path.abspath("__file__"))
sys.path.insert(0, path + '/..')

import base64
import IPython
import importlib
import logging
logging.getLogger().setLevel(logging.ERROR)
import random
import time
from collections import namedtuple

from tf_agents.environments import suite_gym, suite_dm_control, parallel_py_environment
from tf_agents.environments import tf_py_environment, FlattenObservationsWrapper
from tf_agents.metrics import tf_metrics
from tf_agents.replay_buffers import tf_uniform_replay_buffer, episodic_replay_buffer
from tf_agents.drivers import dynamic_episode_driver, dynamic_step_driver
from tf_agents.trajectories import time_step as ts, policy_step, trajectory
from tf_agents.utils import common
from tf_agents.policies import TFPolicy

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)
import tensorflow_probability as tfp
tfd = tfp.distributions

import numpy as np
import json

from reinforcement_learning import labeling_functions
import reinforcement_learning.environments
from reinforcement_learning.environments import EnvironmentLoader, perturbed_env
from reinforcement_learning.metrics import AverageDiscountedReturnMetric
from policies.saved_policy import SavedTFPolicy
from policies.epsilon_mimic import EpsilonMimicPolicy
from policies.latent_policy import LatentPolicyOverRealStateAndActionSpaces

from verification import model, local_losses, binary_latent_space
from verification.value_iteration import value_iteration
from verification.local_losses import compute_values_from_initial_distribution, PolicyDecorator
from util.io.dataset_generator import ergodic_batched_labeling_function, is_reset_state

from typing import Callable, Optional
from tf_agents.typing.types import Float, Bool

from util.io import video
import wasserstein_mdp

# set seed
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

No module named 'reverb' Reverb is not installed on your system, meaning prioritized experience replay cannot be used.


2022-08-09 17:52:54.469084: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-08-09 17:52:54.469110: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: florentdelgrange-MS-7C56
2022-08-09 17:52:54.469114: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: florentdelgrange-MS-7C56
2022-08-09 17:52:54.469170: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 450.203.3
2022-08-09 17:52:54.469185: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 450.172.1
2022-08-09 17:52:54.469188: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 450.172.1 does not match DSO version 450.203.3 -- cannot find working devices in this configuration
2022-08-09 17:52:54.469346: I tensorflow/core/platform/cpu_f

In [2]:
def display_state_space(py_env):
    print("state space shape:", py_env.observation_spec().shape)
    try:
        print("state space max values:", py_env.observation_spec().maximum)
        print("state space min values:", py_env.observation_spec().minimum)
    except AttributeError as e:
        pass

def display_action_space(py_env):
    if py_env.action_spec().dtype in [np.int64, np.int32]:
        print("discrete action space")
        print("number of discrete actions:", py_env.action_spec().maximum + 1)
    else:
        print("continuous action space")
        print("action space shape:", py_env.action_spec().shape)
        print("action space max values:", py_env.action_spec().maximum)
        print("action space min values:", py_env.action_spec().minimum)

In [3]:
from util.io.dataset_generator import is_reset_state
from verification.local_losses import PolicyDecorator

@tf.function
def get_p_init(
    wae_mdp,
    original_state,
    latent_transition_fn,
    environment_name,
):
    latent_state_space = binary_latent_space(wae_mdp.latent_state_size)
    is_reset_state_test_fn = lambda latent_state: is_reset_state(latent_state, wae_mdp.atomic_prop_dims)
    original_reset_state = tf.tile(tf.zeros_like(original_state[:1, ...]), [tf.shape(latent_state_space)[0], 1])
    reset_state = wae_mdp.state_embedding_function(
        original_reset_state,
        ergodic_batched_labeling_function(
            labeling_functions[environment_name]
        )(original_reset_state))
    reset_state = tf.cast(reset_state, tf.float32)
    
    latent_action_space = tf.one_hot(
        indices=tf.range(wae_mdp.number_of_discrete_actions),
        depth=tf.cast(wae_mdp.number_of_discrete_actions, tf.int32),
        dtype=tf.float32)

    return tf.reduce_sum(
        tf.transpose(
            PolicyDecorator(wae_mdp.get_latent_policy(action_dtype=tf.int64))(
                reset_state
            ).probs_parameter()
        ) * tf.map_fn(
            fn=lambda latent_action: latent_transition_fn(
                reset_state,
                tf.tile(tf.expand_dims(latent_action, 0), [tf.shape(latent_state_space)[0], 1]),
            ).prob(
                tf.cast(latent_state_space, tf.float32),
                full_latent_state_space=True),
            elems=latent_action_space),
        axis=0) * (1. - tf.cast(is_reset_state_test_fn(latent_state_space), tf.float32))


In [4]:
def C_until_T_values(
    C_fn: Callable[[Float], Bool],
    T_fn: Callable[[Float], Bool],
    transition_matrix: Float,
    latent_state_size: int,
    A: int,
    latent_policy: TFPolicy,
    gamma: Float = 0.99,
    transition_to_T_reward: Optional[Float] = None,
) -> Float:
    
    S = tf.pow(2, latent_state_size)
    state_space = binary_latent_space(latent_state_size, dtype=tf.float32)
    
    # make absorbing ¬C and T
    absorbing_states = lambda latent_state: tf.math.logical_or(
        tf.math.logical_not(C_fn(latent_state)),
        T_fn(latent_state))
    
    # reward of 1 when transitioning to T;
    # set it to the input values if provided
    reward_objective = tf.ones(
        shape=(S, A, S),
    ) * tf.cast(T_fn(state_space), tf.float32)
    if transition_to_T_reward is not None:
        reward_objective *= transition_to_T_reward
    
    policy_probs = PolicyDecorator(
        latent_policy
    )(state_space).probs_parameter()
    
    values = value_iteration(
        latent_state_size=latent_state_size,
        num_actions=A,
        transition_fn=transition_matrix,
        reward_fn=reward_objective,
        gamma=gamma,
        policy_probs=policy_probs,
        epsilon=1e-6,
        v_init=tf.zeros(S, dtype=tf.float32),
        episodic_return=True,
        is_reset_state_test_fn=absorbing_states,
        error_type='absolute',
        transition_matrix=transition_matrix,
        reward_matrix=reward_objective,)
    
    # set the values of the target states to either one or the input values if provided
    if transition_to_T_reward is None:
        values = values + tf.cast(T_fn(state_space), tf.float32)
    else:
        values = values + (tf.cast(T_fn(state_space), tf.float32) * transition_to_T_reward)
    
    return values

In [5]:
def reach_C_then_T_values(
    C_fn: Callable[[Float], Bool],
    T_fn: Callable[[Float], Bool],
    transition_matrix: Float,
    latent_state_size: int,
    A: int,
    latent_policy: TFPolicy,
    gamma: Float = 0.99,
) -> Float:
    
    S = tf.pow(2, latent_state_size)
    state_space = binary_latent_space(latent_state_size, dtype=tf.float32)

    C = C_fn(state_space)
    T = T_fn(state_space)
    all_states = tf.ones(shape=(S, A, S))
    
    # detect when the agent transitions from the C to T
    # set C-state rows to 1 
    from_C = tf.transpose(all_states * tf.cast(C, tf.float32))
    # set T-state columns to 1 
    to_T = all_states * tf.cast(T, tf.float32)
    C_to_T_transitions = from_C * to_T
    
    # create the MDP augmented by a new absorbing state where C-states transition
    # to instead of transitioning to T-states
    #
    # get the probability of transitioning from C to T
    C_to_T_probs = tf.reduce_sum(transition_matrix * C_to_T_transitions, axis=-1)

    # deviate the transitions from C to a new absorbing state
    augmented_transition_matrix = tf.concat(
        # set the probabilities of transitioning from C to T to 0.
        [transition_matrix * (1. - C_to_T_transitions),
         # set the transition probabilities to the absorbing state to those
         # of transitioning to T
         tf.expand_dims(C_to_T_probs, axis=-1)],
        axis=-1)
    # create a new sink state
    sink_state_probs = tf.concat([
            tf.zeros(shape=(1, A, S)),
            tf.ones(shape=(1, A, 1))
        ], axis=-1)
    # add this sink state to the transition matrix of the augmented MDP
    augmented_transition_matrix = tf.concat([
        augmented_transition_matrix,
        sink_state_probs,
    ], axis=0)

    # enable some random actions for the sink state
    policy_probs = PolicyDecorator(
        latent_policy
    )(state_space).probs_parameter()
    policy_probs = tf.concat([
        policy_probs,
        tf.pow(
            tf.cast(A, tf.float32), -1.
        ) * tf.ones(shape=(1, A))
    ], axis=0)
    
    # reward of 1 when transitioning to the sink state
    reward_objective = tf.concat([
        tf.zeros(shape=(S, A, S)),
        # add a last column full of ones
        tf.ones(shape=(S, A, 1))
    ], axis=-1)
    reward_objective = tf.concat([
        # add a last row full of zeros
        reward_objective,
        tf.zeros(shape=(1, A, S + 1))
    ], axis=0)
    
    return value_iteration(
        latent_state_size=latent_state_size,
        num_actions=A,
        transition_fn=augmented_transition_matrix,
        reward_fn=reward_objective,
        gamma=gamma,
        policy_probs=policy_probs,
        epsilon=1e-6,
        v_init=tf.zeros(S + 1, dtype=tf.float32),
        episodic_return=False,
        error_type='absolute',
        transition_matrix=augmented_transition_matrix,
        reward_matrix=reward_objective,)

## CartPole

In [127]:
wae_model_path = 'saved_models/experiments/CartPole-v0/model/'

with open(os.path.join(wae_model_path, 'model_infos.json'), 'r') as f:
    wae_data = json.load(f)
    print(wae_data)

wae_mdp = wasserstein_mdp.load(wae_model_path)
print("WAE-MDP loaded")

{'self': '<wasserstein_mdp.WassersteinMarkovDecisionProcess object at 0x2addb46fe040>', 'state_shape': '(4,)', 'action_shape': '(2,)', 'reward_shape': '(1,)', 'label_shape': '(2,)', 'discretize_action_space': 'False', 'state_encoder_network': "ModelArchitecture(hidden_units=[64, 64, 64], activation='tanh', name='state_encoder_network_base')", 'action_decoder_network': "ModelArchitecture(hidden_units=[64, 64, 64], activation='tanh', name='action_decoder_network_base')", 'transition_network': "ModelArchitecture(hidden_units=[64, 64, 64], activation='tanh', name='transition_network_base')", 'reward_network': "ModelArchitecture(hidden_units=[64, 64, 64], activation='tanh', name='reward_network_base')", 'decoder_network': "ModelArchitecture(hidden_units=[64, 64, 64], activation='tanh', name='state_decoder_network_base')", 'latent_policy_network': "ModelArchitecture(hidden_units=[64, 64, 64], activation='tanh', name='discrete_policy_network_base')", 'steady_state_lipschitz_network': "ModelAr

In [128]:
print("WAE-MDP at training step {:d}".format(eval(wae_data['training_step'])))
print("Size of the latent state space: {:d}".format(2 ** wae_mdp.latent_state_size))

WAE-MDP at training step 120000
Size of the latent state space: 512


In [129]:
with suite_gym.load('CartPole-v0') as py_env:
    py_env.reset()
    tf_env = tf_py_environment.TFPyEnvironment(py_env)

_latent_transition_fn = lambda latent_state, latent_action: \
        wae_mdp.discrete_latent_transition(
            tf.cast(latent_state, tf.float32),
            tf.cast(latent_action, tf.float32))

print('Local reward loss: {:.2g}'.format(eval(wae_data['local_reward_loss'])))
print('Local transition loss: {:.2g}'.format(eval(wae_data['local_transition_loss'])))

print('Transition/reward model generation')
start = time.time()

#  write the transition/reward functions to tensors,
#  to formally check the values in an efficient way
latent_transition_fn = model.TransitionFunctionCopy(
    num_states=tf.cast(tf.pow(2, wae_mdp.latent_state_size), dtype=tf.int32),
    num_actions=wae_mdp.number_of_discrete_actions,
    transition_function=_latent_transition_fn,
    epsilon=1e-6)

end = time.time() - start

print("Time to generate the model: {:.2g} sec".format(end))

Local reward loss: 0.0038
Local transition loss: 0.4
Transition/reward model generation
Time to generate the model: 0.72 sec


The CartPole angle and position are always safe (along all episodes): $\square \neg \textsf{Unsafe} \equiv \neg \Diamond \textsf{Unsafe}$

In [140]:
unsafe_state_test_fn = lambda latent_state: tf.logical_or(
    # unsafe position
    tf.cast(1. - latent_state[..., 0], tf.bool),
    # unsafe angle
    tf.cast(1. - latent_state[..., 1], tf.bool))
true_fn = lambda latent_state: tf.cast(
    tf.ones(shape=(tf.pow(2, wae_mdp.latent_state_size), )),
    tf.bool)

V = C_until_T_values(
    C_fn=true_fn,
    T_fn=unsafe_state_test_fn,
    transition_matrix=latent_transition_fn.to_dense(),
    latent_state_size=wae_mdp.latent_state_size,
    A=wae_mdp.number_of_discrete_actions,
    latent_policy=wae_mdp.get_latent_policy(action_dtype=tf.int64),
    gamma=0.99,)

# compute the initial distribution
p_init = get_p_init(
    wae_mdp,
    tf_env.current_time_step().observation,
    latent_transition_fn,
    'CartPole-v0',)

# get the values for the initial distribution
latent_mdp_values = tf.reduce_sum(
    p_init * V
) / tf.reduce_sum(p_init)

tf.print("property values: {:.6g}".format(1. - latent_mdp_values))

end = time.time() - start
print("Time to compute the values of the property: {:2g} sec".format(end))

property values: 0.958815
Time to compute the values of the property: 445.844 sec


## MountainCar

In [141]:
wae_model_path = 'saved_models/experiments/MountainCar-v0/model/'

with open(os.path.join(wae_model_path, 'model_infos.json'), 'r') as f:
    wae_data = json.load(f)
    print(wae_data)

wae_mdp = wasserstein_mdp.load(wae_model_path)

print("WAE-MDP loaded")

{'self': '<wasserstein_mdp.WassersteinMarkovDecisionProcess object at 0x2b12ba1fbee0>', 'state_shape': '(2,)', 'action_shape': '(3,)', 'reward_shape': '(1,)', 'label_shape': '(3,)', 'discretize_action_space': 'False', 'state_encoder_network': "ModelArchitecture(hidden_units=[512, 512, 512], activation='relu', name='state_encoder_network_base')", 'action_decoder_network': "ModelArchitecture(hidden_units=[512, 512, 512], activation='relu', name='action_decoder_network_base')", 'transition_network': "ModelArchitecture(hidden_units=[512, 512, 512], activation='relu', name='transition_network_base')", 'reward_network': "ModelArchitecture(hidden_units=[512, 512, 512], activation='relu', name='reward_network_base')", 'decoder_network': "ModelArchitecture(hidden_units=[512, 512, 512], activation='relu', name='state_decoder_network_base')", 'latent_policy_network': "ModelArchitecture(hidden_units=[512, 512, 512], activation='relu', name='discrete_policy_network_base')", 'steady_state_lipschitz_

In [142]:
print("WAE-MDP at training step {:d}".format(eval(wae_data['training_step'])))
print("Size of the latent state space: {:d}".format(2 ** wae_mdp.latent_state_size))
print('Local reward loss: {:.6g}'.format(eval(wae_data['local_reward_loss'])))
print('Local transition loss: {:.6g}'.format(eval(wae_data['local_transition_loss'])))

WAE-MDP at training step 840000
Size of the latent state space: 1024
Local reward loss: 0.00577215
Local transition loss: 0.426904


In [143]:
with suite_gym.load('MountainCar-v0') as py_env:
    py_env.reset()
    tf_env = tf_py_environment.TFPyEnvironment(py_env)
    original_state = tf_env.current_time_step().observation

print('Transition/reward model generation')
start = time.time()

_latent_transition_fn = lambda latent_state, latent_action: \
        wae_mdp.discrete_latent_transition(
            tf.cast(latent_state, tf.float32),
            tf.cast(latent_action, tf.float32))

latent_transition_fn = model.TransitionFunctionCopy(
    num_states=tf.cast(tf.pow(2, wae_mdp.latent_state_size), dtype=tf.int32),
    num_actions=wae_mdp.number_of_discrete_actions,
    transition_function=_latent_transition_fn,
    epsilon=1e-6)

end = time.time() - start

print("Time to generate the model: {:.2g} sec".format(end))

Transition/reward model generation
Time to generate the model: 9.1 sec


The car reaches the flag infinitly often: $\Box \Diamond \textsf{Flag} \equiv \neg \Diamond \neg (\Diamond \textsf{Flag}) = \neg \Diamond (\Box \neg \textsf{Flag})$

In [147]:
start = time.time()

flag_fn = lambda latent_state: tf.cast(latent_state[..., 0], tf.bool)
reset_state_fn = lambda latent_state: is_reset_state(latent_state, wae_mdp.atomic_prop_dims)
true_fn = lambda latent_state: tf.cast(
    tf.ones(shape=(tf.pow(2, wae_mdp.latent_state_size), )),
    tf.bool)

# always operator (always not Flag)
V_1 = C_until_T_values(
    C_fn=true_fn,
    T_fn=flag_fn,
    transition_matrix=latent_transition_fn.to_dense(),
    latent_state_size=wae_mdp.latent_state_size,
    A=wae_mdp.number_of_discrete_actions,
    latent_policy=wae_mdp.get_latent_policy(action_dtype=tf.int64),
    gamma=0.9999,)
V_1 = 1. - V_1

# eventually operator (eventually not flag)
V_2 = C_until_T_values(
    C_fn=true_fn,
    T_fn=lambda latent_state: tf.logical_not(flag_fn(latent_state)),
    transition_matrix=latent_transition_fn.to_dense(),
    latent_state_size=wae_mdp.latent_state_size,
    A=wae_mdp.number_of_discrete_actions,
    latent_policy=wae_mdp.get_latent_policy(action_dtype=tf.int64),
    gamma=0.9999,
    transition_to_T_reward=V_1)

# eventually not flag, and then always not flag
# negation
V_3 = 1. - V_2

# compute the initial distribution
p_init = get_p_init(
    wae_mdp,
    tf_env.current_time_step().observation,
    latent_transition_fn,
    'MountainCar-v0',)

# get the values for the initial distribution
latent_mdp_values = tf.reduce_sum(
    p_init * V_3
) / tf.reduce_sum(p_init)

tf.print("property values: {:.6g}".format(latent_mdp_values))

end = time.time() - start
print("Time to compute the values of the property: {:2g} sec".format(end))

property values: 0.893684
Time to compute the values of the property: 15.3366 sec


## Pendulum

In [6]:
wae_model_path = 'saved_models/experiments/PendulumRandomInit-v1/model/'

with open(os.path.join(wae_model_path, 'model_infos.json'), 'r') as f:
    wae_data = json.load(f)
    print(wae_data)

wae_mdp = wasserstein_mdp.load(wae_model_path)

print("WAE-MDP loaded")

{'self': '<wasserstein_mdp.WassersteinMarkovDecisionProcess object at 0x2ba6945ccf10>', 'state_shape': '(3,)', 'action_shape': '(1,)', 'reward_shape': '(1,)', 'label_shape': '(4,)', 'discretize_action_space': 'True', 'state_encoder_network': "ModelArchitecture(hidden_units=[256, 256, 256], activation='relu', name='state_encoder_network_base')", 'action_decoder_network': "ModelArchitecture(hidden_units=[256, 256, 256], activation='relu', name='action_decoder_network_base')", 'transition_network': "ModelArchitecture(hidden_units=[256, 256, 256], activation='relu', name='transition_network_base')", 'reward_network': "ModelArchitecture(hidden_units=[256, 256, 256], activation='relu', name='reward_network_base')", 'decoder_network': "ModelArchitecture(hidden_units=[256, 256, 256], activation='relu', name='state_decoder_network_base')", 'latent_policy_network': "ModelArchitecture(hidden_units=[256, 256, 256], activation='relu', name='discrete_policy_network_base')", 'steady_state_lipschitz_n

2022-08-09 17:53:07.911364: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Model: "state_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 state (InputLayer)          [(None, 3)]               0         
                                                                 
 state_encoder_body (Sequent  (None, 256)              132608    
 ial)                                                            
                                                                 
 dense_5 (Dense)             (None, 8)                 2056      
                                                                 
Total params: 134,664
Trainable params: 134,664
Non-trainable params: 0
_________________________________________________________________
Model: "action_encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 latent_state (InputLay

In [7]:
print("WAE-MDP at training step {:d}".format(eval(wae_data['training_step'])))
print("Size of the latent state space: {:d}".format(2 ** wae_mdp.latent_state_size))
print("Size of the latent action space: {:d}".format(wae_mdp.number_of_discrete_actions))
print('Local reward loss: {:.6g}'.format(eval(wae_data['local_reward_loss'])))
print('Local transition loss: {:.6g}'.format(eval(wae_data['local_transition_loss'])))

WAE-MDP at training step 370000
Size of the latent state space: 8192
Size of the latent action space: 3
Local reward loss: 0.0266745
Local transition loss: 0.539508


In [8]:
with suite_gym.load('Pendulum-v1') as py_env:
    py_env.reset()
    tf_env = tf_py_environment.TFPyEnvironment(py_env)
    original_state = tf_env.current_time_step().observation

_latent_transition_fn = lambda latent_state, latent_action: \
        wae_mdp.discrete_latent_transition(
            tf.cast(latent_state, tf.float32),
            tf.cast(latent_action, tf.float32))

print('Local reward loss: {:.2g}'.format(eval(wae_data['local_reward_loss'])))
print('Local transition loss: {:.2g}'.format(eval(wae_data['local_transition_loss'])))

print('Transition/reward model generation')
#  write the transition/reward functions to tensors,
#  to formally check the values in an efficient way
start = time.time()

latent_transition_fn = model.TransitionFunctionCopy(
    num_states=tf.cast(tf.pow(2, wae_mdp.latent_state_size), dtype=tf.int32),
    num_actions=wae_mdp.number_of_discrete_actions,
    transition_function=_latent_transition_fn,
    epsilon=1e-9)

end = time.time() - start

print("Time to generate the model: {:.2f} sec".format(end))

Local reward loss: 0.027
Local transition loss: 0.54
Transition/reward model generation
Time to generate the model: 217.49 sec


There is an episode where the pendulum straightens and then remains straight until the end of the episode: $\Diamond [\textsf{Upright}\, \mathcal{U} \textsf{Reset}]$

In [114]:
start = time.time()

upright_test_fn = lambda latent_state: tf.cast(latent_state[..., 0], tf.bool)
reset_state_test_fn = lambda latent_state: is_reset_state(latent_state, wae_mdp.atomic_prop_dims)

V_1 = reach_C_then_T_values(
    C_fn=upright_test_fn,
    T_fn=reset_state_test_fn,
    transition_matrix=latent_transition_fn.to_dense(),
    latent_state_size=wae_mdp.latent_state_size,
    A=wae_mdp.number_of_discrete_actions,
    latent_policy=wae_mdp.get_latent_policy(action_dtype=tf.int64),
    gamma=0.9999)

# compute the initial distribution
p_init = get_p_init(
    wae_mdp,
    tf_env.current_time_step().observation,
    latent_transition_fn,
    'Pendulum-v1',)
# take into account the added absorbing state
p_init = tf.concat([p_init, [0.]], axis=-1)

# get the values for the initial distribution
latent_mdp_values = tf.reduce_sum(
    p_init * V
) / tf.reduce_sum(p_init)

tf.print("property values: {:.6g}".format(latent_mdp_values))

end = time.time() - start
print("Time to compute the values of the property: {:2g} sec".format(end))

property values: 0.948295
Time to compute the values of the property: 658.447 sec


The pendulum always straightens and then remains it until the end of the current episode: $\Box\Diamond(\textsf{Upright}\mathcal{U}\textsf{Reset}) \equiv \neg \Diamond \neg [ \Diamond(\textsf{Upright}\mathcal{U}\textsf{Reset}) ] \equiv \neg \Diamond \Box \neg (\textsf{Upright}\mathcal{U}\textsf{Reset})$

In [115]:
true_fn = lambda latent_state: tf.cast(
    tf.ones(shape=(tf.pow(2, wae_mdp.latent_state_size), )),
    tf.bool)

# always operator: always not upright until reset
# = not eventually upright until reset
V_1 = 1 - V

# eventually operator: eventually upright and not reset,
# then always not (upright until reset)
V_2 = C_until_T_values(
    C_fn=true_fn,
    # cf. negation of not (upright until reset)
    T_fn=lambda latent_state: tf.logical_and(
        upright_test_fn(latent_state),
        tf.logical_not(reset_state_test_fn(latent_state))),
    transition_matrix=latent_transition_fn.to_dense(),
    latent_state_size=wae_mdp.latent_state_size,
    A=wae_mdp.number_of_discrete_actions,
    latent_policy=wae_mdp.get_latent_policy(action_dtype=tf.int64),
    gamma=0.9999,
    # the reward of transitioning to (absorbing) T is the probability
    # of always not (upright until reset), recorded in V_1
    transition_to_T_reward=V_1[:-1] 
)

# negation of eventually always not (upright until reset)
V_3 = 1 - V_2

# compute the initial distribution
p_init = get_p_init(
    wae_mdp,
    tf_env.current_time_step().observation,
    latent_transition_fn,
    'Pendulum-v1',)

# get the values for the initial distribution
latent_mdp_values = tf.reduce_sum(
    p_init * V_3
) / tf.reduce_sum(p_init)

tf.print("property values: {:.6g}".format(latent_mdp_values))

end = time.time() - start
print("Time to compute the values of the property: {:2g} sec".format(end))

property values: 0.953257
Time to compute the values of the property: 697.149 sec


## LunarLander

In [151]:
wae_model_path = 'saved_models/experiments/LunarLanderContinuous-v2/model/'

with open(os.path.join(wae_model_path, 'model_infos.json'), 'r') as f:
    wae_data = json.load(f)
    print(wae_data)

wae_mdp = wasserstein_mdp.load(wae_model_path)

print("WAE-MDP loaded")

{'self': '<wasserstein_mdp.WassersteinMarkovDecisionProcess object at 0x2b74bebf5040>', 'state_shape': '(8,)', 'action_shape': '(2,)', 'reward_shape': '(1,)', 'label_shape': '(6,)', 'discretize_action_space': 'True', 'state_encoder_network': "ModelArchitecture(hidden_units=[256], activation='relu', name='state_encoder_network_base')", 'action_decoder_network': "ModelArchitecture(hidden_units=[256], activation='relu', name='action_decoder_network_base')", 'transition_network': "ModelArchitecture(hidden_units=[256], activation='relu', name='transition_network_base')", 'reward_network': "ModelArchitecture(hidden_units=[256], activation='relu', name='reward_network_base')", 'decoder_network': "ModelArchitecture(hidden_units=[256], activation='relu', name='state_decoder_network_base')", 'latent_policy_network': "ModelArchitecture(hidden_units=[256], activation='relu', name='discrete_policy_network_base')", 'steady_state_lipschitz_network': "ModelArchitecture(hidden_units=[256], activation='

In [152]:
print("WAE-MDP at training step {:d}".format(eval(wae_data['training_step'])))
print("Size of the latent state space: {:d}".format(2 ** wae_mdp.latent_state_size))
print("Size of the latent action space: {:d}".format(wae_mdp.number_of_discrete_actions))
print('Local reward loss: {:.6g}'.format(eval(wae_data['local_reward_loss'])))
print('Local transition loss: {:.6g}'.format(eval(wae_data['local_transition_loss'])))

WAE-MDP at training step 320000
Size of the latent state space: 16384
Size of the latent action space: 3
Local reward loss: 0.0207205
Local transition loss: 0.131357


In [153]:
with suite_gym.load('LunarLanderContinuous-v2') as py_env:
    py_env.reset()
    tf_env = tf_py_environment.TFPyEnvironment(py_env)
    original_state = tf_env.current_time_step().observation

start = time.time()

with tf.device('/CPU:0'):
    _latent_transition_fn = lambda latent_state, latent_action: \
        wae_mdp.discrete_latent_transition(
            tf.cast(latent_state, tf.float32),
            tf.cast(latent_action, tf.float32))
    latent_transition_fn = model.TransitionFunctionCopy(
        num_states=tf.cast(tf.pow(2, wae_mdp.latent_state_size), dtype=tf.int32),
        num_actions=wae_mdp.number_of_discrete_actions,
        transition_function=_latent_transition_fn,
        epsilon=1e-6)

end = time.time() - start
print("Time to generate the model: {:.2f} sec".format(end))

Time to generate the model: 380.14 sec


The lunar lander never crashes along episodes: it never passes from an unsafe landing state to the reset state, i.e., $\Box [\neg (\neg \textsf{SafeLanding} \wedge \bigcirc \textsf{Reset})] \equiv \neg \Diamond (\neg \textsf{SafeLanding} \wedge \bigcirc \textsf{Reset})$

In [ ]:
start = time.time()

safe_landing_fn = lambda latent_state: tf.logical_and(
    tf.cast(latent_state[..., 1], tf.bool),
    tf.cast(latent_state[..., 5], tf.bool))
unsafe_landing_fn = lambda latent_state: tf.logical_not(safe_landing_fn(latent_state))
reset_state_fn = lambda latent_state: is_reset_state(latent_state, wae_mdp.atomic_prop_dims)

V = reach_C_then_T_values(
    C_fn=unsafe_landing_fn,
    T_fn=reset_state_fn,
    transition_matrix=latent_transition_fn.to_dense(),
    latent_state_size=wae_mdp.latent_state_size,
    A=wae_mdp.number_of_discrete_actions,
    latent_policy=wae_mdp.get_latent_policy(action_dtype=tf.int64),
    gamma=0.99)

# compute the initial distribution
p_init = get_p_init(
    wae_mdp,
    tf_env.current_time_step().observation,
    latent_transition_fn,
    'LunarLanderContinuous-v2',)
# take into account the added absorbing state
p_init = tf.concat([p_init, [0.]], axis=-1)

# get the values for the initial distribution
latent_mdp_values = tf.reduce_sum(
    p_init * V
) / tf.reduce_sum(p_init)

tf.print("property values: {:.6g}".format(1. - latent_mdp_values))

end = time.time() - start
print("Time to compute the values of the property: {:2g} sec".format(end))

There is an episode where the angle of the lander is unsafe during the landing:
$\Diamond [\textsf{Unsafe} \, \mathcal{U}\, \textsf{LegContact}]$

In [ ]:
start = time.time()

unsafe_angle_fn = lambda latent_state: tf.cast(latent_state[..., 0], tf.bool)
leg_contact_fn = lambda latent_state: tf.cast(latent_state[..., 1], tf.bool)


V = reach_C_then_T_values(
    C_fn=unsafe_angle_fn,
    T_fn=leg_contact_fn,
    transition_matrix=latent_transition_fn.to_dense(),
    latent_state_size=wae_mdp.latent_state_size,
    A=wae_mdp.number_of_discrete_actions,
    latent_policy=wae_mdp.get_latent_policy(action_dtype=tf.int64),
    gamma=0.99)

# compute the initial distribution
p_init = get_p_init(
    wae_mdp,
    tf_env.current_time_step().observation,
    latent_transition_fn,
    'LunarLanderContinuous-v2',)
# take into account the added absorbing state
p_init = tf.concat([p_init, [0.]], axis=-1)

# get the values for the initial distribution
latent_mdp_values = tf.reduce_sum(
    p_init * V
) / tf.reduce_sum(p_init)

tf.print("property values: {:.6g}".format(latent_mdp_values))

end = time.time() - start
print("Time to compute the values of the property: {:2g} sec".format(end))

In all episodes, the angle of the lander is safe until it lands: $\Box (\textsf{SafeAngle}\, \mathcal{U} \, \textsf{LegContact}) \equiv \neg \Diamond [(\textsf{SafeAngle} \wedge \neg \textsf{LegContact}) \, \mathcal{U}\, (\neg \textsf{SafeAngle} \wedge \neg \textsf{LegContact}) \vee \Box(\textsf{SafeAngle} \wedge \neg \textsf{LegContact})]$

In [ ]:
start = time.time()

safe_angle_fn = lambda latent_state: tf.math.logical_not(tf.cast(latent_state[..., 0], tf.bool))
leg_contact_fn = lambda latent_state: tf.cast(latent_state[..., 1], tf.bool)
true_fn = lambda latent_state: tf.cast(
    tf.ones(shape=(tf.pow(2, wae_mdp.latent_state_size), )),
    tf.bool)

# until operator
# (discounted) probability of being (safe and not leg contact)
# until (not safe and not leg contact)
V_1 = C_until_T_values(
    C_fn=lambda latent_state: tf.logical_and(
        safe_angle_fn(latent_state),
        tf.logical_not(leg_contact_fn(latent_state))),
    T_fn=lambda latent_state: tf.logical_and(
        tf.logical_not(safe_angle_fn(latent_state)),
        tf.logical_not(leg_contact_fn(latent_state))),
    transition_matrix=latent_transition_fn.to_dense(),
    latent_state_size=wae_mdp.latent_state_size,
    A=wae_mdp.number_of_discrete_actions,
    latent_policy=wae_mdp.get_latent_policy(action_dtype=tf.int64),
    gamma=0.99)

# always operator
V_2 = C_until_T_values(
    C_fn=true_fn,
    T_fn=lambda latent_state: tf.logical_or(
        tf.logical_not(safe_angle_fn(latent_state)),
        leg_contact_fn(latent_state)),
    transition_matrix=latent_transition_fn.to_dense(),
    latent_state_size=wae_mdp.latent_state_size,
    A=wae_mdp.number_of_discrete_actions,
    latent_policy=wae_mdp.get_latent_policy(action_dtype=tf.int64),
    gamma=0.99)
# discounted probability of always (safe and not leg contact)
V_2 = 1. - V_2

# OR operator
V_3 = tf.reduce_max(
    tf.concat([
        tf.expand_dims(V_1, -1),
        tf.expand_dims(V_2, -1)],
        axis=-1),
    axis=-1)

# eventually operator
# eventually reach (safe and not leg contact)
V_4 = C_until_T_values(
    C_fn=true_fn,
    T_fn=lambda latent_state: tf.logical_and(
        safe_angle_fn(latent_state),
        tf.logical_not(leg_contact_fn(latent_state))),
    transition_matrix=latent_transition_fn.to_dense(),
    latent_state_size=wae_mdp.latent_state_size,
    A=wae_mdp.number_of_discrete_actions,
    latent_policy=wae_mdp.get_latent_policy(action_dtype=tf.int64),
    gamma=0.99,
    # set the rewards of transitioning to (absorbing) T 
    # to the discounted probabilities of the event recorded in V_3
    transition_to_T_reward=V_3
)

# Final property values
V_prop = 1. - V_4

# compute the initial distribution
p_init = get_p_init(
    wae_mdp,
    tf_env.current_time_step().observation,
    latent_transition_fn,
    'LunarLanderContinuous-v2',)

# get the values for the initial distribution
latent_mdp_values = tf.reduce_sum(
    p_init * V_prop
) / tf.reduce_sum(p_init)

tf.print("property values: {:.6g}".format(latent_mdp_values))

end = time.time() - start
print("Time to compute the values of the property: {:2g} sec".format(end))